<img src="https://10botics.com/logo_jnb.png" width="300"/>

# Applying model to a video

## Include libraries and functions

In [ ]:
%run Squat_common_v2.ipynb
%run functions_angle_calculation.ipynb
%run functions_video.ipynb

## Playing a video

In [ ]:
# this frame_process does nothing for now, just returning the frame

def frame_process(frame):
    return frame

In [ ]:
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton, "./resource/Produce_2_direct.mp4", 0, frame_process))
thread.start()


## Add some text to the video

In [ ]:
def putText(frame, text, color = (0, 255, 0), position_x = 50, position_y = 50):
    # Define the text properties
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_position = (position_x, position_y)
    text_scale = 0.65
    text_color = color
    text_thickness = 2

    # Add text annotation on the frame
    cv2.putText(frame, text, text_position, font, text_scale, text_color, text_thickness)

In [ ]:
def frame_process(frame):
    text = f"HAHAHA"
    putText(frame, text)
    return frame

display(stopButton)
thread = threading.Thread(target=view, args=(stopButton, "./resource/Produce_2_direct.mp4", 0, frame_process))
thread.start()


### Add a counter and angles to the video

In [ ]:
count = 0
knee_angle = 0
hip_angle = 0

def frame_process(frame):
    global count
    text = f"Count: {count}  Knee: {int(knee_angle)}  Hip: {int(hip_angle)}"
    count = count + 1
    
    putText(frame, text)
    return frame

display(stopButton)
thread = threading.Thread(target=view, args=(stopButton, "./resource/Produce_2_direct.mp4", 0, frame_process))
thread.start()


## Calculate angle for each frame

In [ ]:
def frame_process(frame):
    global repetition, squat
    # calculate and display angle
    # keypoints_with_scores = get_keypoints_with_scores_from_image_with_movenet(frame,input_size,input_size)

    movenet_frame = convert_to_movenet_format(frame)
    movenet_output = movenet(movenet_frame)
    keypoints_with_scores = reoganize_output(movenet_output)

    hip_angle = calculate_hip_joint_angle(keypoints_with_scores)
    knee_angle = calculate_knee_joint_angle(keypoints_with_scores)


In [ ]:
def resize_with_pad(image, target_height=None, target_width=None, pad_color=(0, 0, 0)):    
    original_height, original_width = image.shape[:2]

    if ((target_height is None) or (target_width is None)):
        target_height = max(original_height, original_width)
        target_width = target_height
    
    
    # Calculate the scaling factor and new dimensions while maintaining aspect ratio
    scale = min(target_width / original_width, target_height / original_height)
    new_width = int(original_width * scale)
    new_height = int(original_height * scale)
    
    # Resize the image
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
    
    # Create a new image with the target dimensions and the padding color
    padded_image = np.full((target_height, target_width, 3), pad_color, dtype=np.uint8)
    
    # Calculate the padding offsets
    x_offset = (target_width - new_width) // 2
    y_offset = (target_height - new_height) // 2
    
    # Place the resized image in the center of the padded image
    padded_image[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_image
    
    return padded_image


In [ ]:
repetition = 0
squat = 0

def absolute_xy(frame, point):
    x = point[1] * frame.shape[1]
    y = point[0] * frame.shape[0]
    return (int(x), int(y))
    
def frame_process(frame):
    global repetition, squat
    # calculate and display angle
    # keypoints_with_scores = get_keypoints_with_scores_from_image_with_movenet(frame,input_size,input_size)

    movenet_frame = convert_to_movenet_format(frame)
    movenet_output = movenet(movenet_frame)
    keypoints_with_scores = reoganize_output(movenet_output)

    hip_angle = calculate_hip_joint_angle(keypoints_with_scores)
    knee_angle = calculate_knee_joint_angle(keypoints_with_scores)

    frame = resize_with_pad(frame)
    # frame = movenet_frame.numpy()
    # print(type(movenet_frame[0]))
    # print(movenet_frame[0])
    # frame = movenet_frame[0].numpy()


    # Gesture judgement from angle of hip-joint and knee-joint
    # if gesture is from squat (squat == 1) change to stand (squat == 0), repetition add by 1
    if hip_angle<135 and knee_angle<120:
        squat = 1
    elif hip_angle>135 and knee_angle>120:
        if squat == 1:
            repetition += 1
        squat = 0
    text = f"Reps: {repetition}  Knee: {int(knee_angle)}  Hip: {int(hip_angle)}"
    if squat:
        text_color = (0,255,0)
    else:
        text_color = (128, 192, 64)

    putText(frame, text, text_color)

    return frame


# Run
# ================

display(stopButton)
thread = threading.Thread(target=view, args=(stopButton, "./resource/Produce_2_direct.mp4", 0, frame_process, 10))
thread.start()



## Annotate the keypoints

In [ ]:
# Define the color (BGR) and size

def cv2_circle(frame, xy, text=None):
    
    color = (255, 0, 0)  # Blue color
    radius = 5
    thickness = -1  # Solid circle
    
    # Draw the point
    cv2.circle(frame, xy, radius, color, thickness)

    if text is not None:
        x,y = xy
        
        putText(frame, text, (255,0,0),  int(x * 1.1), y)
        

In [ ]:
repetition = 0
squat = 0

def frame_process(frame):
    global repetition, squat
    # calculate and display angle
    # keypoints_with_scores = get_keypoints_with_scores_from_image_with_movenet(frame,input_size,input_size)

    movenet_frame = convert_to_movenet_format(frame)
    movenet_output = movenet(movenet_frame)
    keypoints_with_scores = reoganize_output(movenet_output)

    hip_angle = calculate_hip_joint_angle(keypoints_with_scores)
    knee_angle = calculate_knee_joint_angle(keypoints_with_scores)

    frame = resize_with_pad(frame)
    # frame = movenet_frame.numpy()
    # print(type(movenet_frame[0]))
    # print(movenet_frame[0])
    # frame = movenet_frame[0].numpy()


    # Gesture judgement from angle of hip-joint and knee-joint
    # if gesture is from squat (squat == 1) change to stand (squat == 0), repetition add by 1
    if hip_angle<135 and knee_angle<120:
        squat = 1
    elif hip_angle>135 and knee_angle>120:
        if squat == 1:
            repetition += 1
        squat = 0
    text = f"Reps: {repetition}  Knee: {int(knee_angle)}  Hip: {int(hip_angle)}"
    if squat:
        text_color = (0,255,0)
    else:
        text_color = (128, 192, 64)

    # # draw visualized prediction from MoveNet on the frame. Attention: Not recommended, VERY LAGGY!
    # #frame = draw_prediction_on_image( tf_image_to_model(frame,600,600)[0]/255, keypoints_with_scores)
    
    
    putText(frame, text, text_color)
    # hip_x, hip_y = 
    # hip_x, hip_y = absolute_xy(frame, keypoints_with_scores['right']['shoulder'])

    for body_part in body_parts:    
        cv2_circle(frame, absolute_xy(frame, keypoints_with_scores['right'][body_part]), body_part)
    
    # cv2_circle(frame, absolute_xy(frame, keypoints_with_scores['right']['shoulder']))
    # cv2_circle(frame, absolute_xy(frame, keypoints_with_scores['right']['knee']))
    # cv2_circle(frame, absolute_xy(frame, keypoints_with_scores['right']['ankle']))
    
    
    putText(frame, "hip", text_color)
    

    return frame


# Run
# ================

display(stopButton)
thread = threading.Thread(target=view, args=(stopButton, "./resource/Produce_2_direct.mp4", 0, frame_process, 10))
thread.start()



<hr/>

## Congratulation! You have finished this chapter.

This jupyter notebook is created by 10Botics. <br>
For permission to use in school, please contact info@10botics.com <br>
All rights reserved. 2024.